In [2]:
import geopandas as gpd
from glob import glob
from shapely.ops import cascaded_union#, unary_union
from shapely.geometry import mapping
import h3
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
# import
import warnings
warnings.filterwarnings('ignore')
assets = '/app/assets/'

In [3]:
# Función para obtener hexágonos H3 dentro de un polígono
def obtener_hexagonos_en_gdf(poligono, resolucion, expansion=0.002):
    # Expande el polígono ligeramente para asegurar cobertura completa
    poligono_expandido = poligono.buffer(expansion)
    
    # Convertir el polígono expandido a un formato que h3.polyfill pueda entender
    contorno_expandido = mapping(poligono_expandido)['coordinates'][0]
    contorno_expandido = [(y, x) for x, y in contorno_expandido]  # Asegúrate de que las coordenadas estén en formato (lat, lon)
    
    # Obtener los hexágonos usando el contorno del polígono expandido
    hexagonos = h3.polyfill_geojson({"type": "Polygon", "coordinates": [contorno_expandido]}, resolucion)
    
    return list(hexagonos)

# Función para convertir un código H3 a un objeto Polygon de Shapely, modificada para devolver también el código H3
def h3_to_polygon(hex_code):
    # Obtener los límites del hexágono como una lista de coordenadas (lat, lon)
    boundary = h3.h3_to_geo_boundary(hex_code, geo_json=True)
    # Invertir las coordenadas para cada punto del límite
    boundary_inverted = [(lon, lat) for lat, lon in boundary]
    # Crear y retornar un objeto Polygon usando estas coordenadas invertidas
    return Polygon(boundary_inverted), hex_code

def get_h3_set(gdf, resolucion):
    gdf['geometry'] = gdf['geometry'].apply(lambda poly: Polygon([(x, y) for x, y, z in poly.exterior.coords]))
    original_crs = gdf.crs
    # Suponiendo que ya has corregido la geometría de 'gdf' para que solo contenga coordenadas x, y
    # Aplicar la función al polígono en tu GeoDataFrame para obtener los hexágonos

    gdf = gdf.to_crs(4326)
    gdf['hexagonos_h3'] = gdf['geometry'].apply(lambda poly: obtener_hexagonos_en_gdf(poly, resolucion))

    # Suponiendo que 'hexagonos_h3' es una lista de códigos H3 obtenidos previamente y 'resolucion' es conocida
    hexagonos_h3 = gdf['hexagonos_h3'].explode().tolist()  # Aplanamos la lista si es necesario

    # Convertir cada código H3 a un Polygon y mantener el código H3
    poligonos_y_codigos = [h3_to_polygon(hex_code) for hex_code in hexagonos_h3]

    # Separar los polígonos y los códigos H3 en listas separadas
    poligonos, codigos_h3 = zip(*poligonos_y_codigos)

    # Crear un DataFrame con los códigos H3 y la resolución, luego convertirlo a GeoDataFrame
    df_hexagonos = pd.DataFrame({
        'geometry': poligonos,
        'resolucion': [resolucion] * len(codigos_h3),
        'hex_id': codigos_h3
    })
    gdf_hexagonos = gpd.GeoDataFrame(df_hexagonos, geometry='geometry')

    overlay_hexs = gpd.overlay(gdf_hexagonos, gdf)
    filters_hexs = gdf_hexagonos[gdf_hexagonos['hex_id'].isin(overlay_hexs['hex_id'])]

    overlay_hexs = overlay_hexs.set_crs(4326)
    filters_hexs = filters_hexs.set_crs(4326)
    return overlay_hexs, filters_hexs

In [24]:
area = gpd.read_parquet(f'{assets}/area_scope/area_scope.parquet')
resolucion = 13
overlay_hexs, filters_hexs = get_h3_set(area.copy(), resolucion)
hex_cols = ['hex_id', 'geometry']
filters_hexs = filters_hexs[hex_cols]
filters_hexs.to_parquet(f'/app/data/set/hex/level_{resolucion}.parquet')